In [ ]:
import pandas as pd
import numpy as np
from joblib import load, dump
from copy import deepcopy
from statistics import mean

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from collections import Counter

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/output/Features.csv')

In [ ]:
df

,product,answer_option,label,review_len,Rn,Rp,Rs,Rc,Rd,Rsc
0,Accucheck,Fast and accurate delivery,0,4,0.233709,0.300000,0.616667,0.005195,1.0,0.0000
1,Accucheck,Expected a longer expire date. Your Product Li...,0,14,0.673415,-0.100000,0.400000,0.016883,1.0,0.0000
2,Accucheck,I liked the prompt service,0,5,0.319747,0.600000,0.800000,0.006494,1.0,0.4215
3,Accucheck,Good product,0,2,0.546925,0.700000,0.600000,0.002597,0.0,0.4404
4,Accucheck,I not needed,0,3,0.000000,0.000000,0.000000,0.003896,1.0,0.0000
...,...,...,...,...,...,...,...,...,...,...
1650,shampoo,Its not much effective as it has been stated i...,0,12,0.500000,-0.300000,0.800000,0.028640,0.0,-0.3724
1651,shampoo,Liked it very nicely working now my scalp is a...,1,11,0.166375,0.690000,0.900000,0.026253,0.0,0.5709
1652,shampoo,its my regular choice,0,4,0.500000,0.000000,0.076923,0.009547,0.0,0.0000
1653,shampoo,Good but not very effective,0,5,0.000000,0.234615,0.607692,0.011933,0.0,-0.4032


In [ ]:
data_split = pd.crosstab(df['product'],df['label'])
data_split

label,0,1
product,,
Accucheck,310,85
Becadexamin,53,27
Evion,89,33
Neurobion,280,136
SevenseascodLiverOil,59,22
Shelcal,259,124
Supradyn,50,23
shampoo,56,49


In [ ]:
def building_training_data(df):
    A = df[df['label']==1]
    A.loc[df['label']==1,'join'] = 'j'
    B = df[df['label']==0]
    B.loc[df['label']==0,'join'] = 'j'
    trainset1 = pd.merge(A,B,how='outer',on='join')
    trainset2 = pd.merge(B,A,how='outer',on ='join')

    trainset = pd.merge(trainset1,trainset2,how='outer')
    return trainset

In [ ]:
product_list = df['product'].unique()
data_stack = []
for product in product_list:
    temp = deepcopy(df[df['product']==product].iloc[:,2:])
    build_data = building_training_data(temp)
    print(product, len(temp), len(build_data))
    build_data.drop(columns = ['join','label_y'],inplace=True)
    data = build_data.iloc[:,1:]
    data['target'] = build_data.iloc[:,0]
    data_stack.append(data)

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Accucheck 395 52700
Becadexamin 80 2862
Evion 122 5874
Neurobion 416 76160
SevenseascodLiverOil 81 2596
Shelcal 383 64232
Supradyn 73 2300
shampoo 105 5488


In [ ]:
train = pd.concat(data_stack).reset_index(drop = True)

In [ ]:
train

,review_len_x,Rn_x,Rp_x,Rs_x,Rc_x,Rd_x,Rsc_x,review_len_y,Rn_y,Rp_y,Rs_y,Rc_y,Rd_y,Rsc_y,target
0,5,0.544357,0.52,0.823333,0.006494,0.0,0.0000,4,0.233709,0.300000,0.616667,0.005195,1.0,0.0000,1
1,5,0.544357,0.52,0.823333,0.006494,0.0,0.0000,14,0.673415,-0.100000,0.400000,0.016883,1.0,0.0000,1
2,5,0.544357,0.52,0.823333,0.006494,0.0,0.0000,5,0.319747,0.600000,0.800000,0.006494,1.0,0.4215,1
3,5,0.544357,0.52,0.823333,0.006494,0.0,0.0000,2,0.546925,0.700000,0.600000,0.002597,0.0,0.4404,1
4,5,0.544357,0.52,0.823333,0.006494,0.0,0.0000,2,0.546925,0.700000,0.600000,0.002597,0.0,0.4404,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212207,10,0.413633,0.50,0.500000,0.023866,1.0,-0.1263,15,0.689470,0.700000,0.600000,0.035800,1.0,0.1531,0
212208,10,0.413633,0.50,0.500000,0.023866,1.0,-0.1263,7,0.392391,0.316667,0.600000,0.016706,0.0,0.2382,0
212209,10,0.413633,0.50,0.500000,0.023866,1.0,-0.1263,25,0.575335,0.233333,0.255556,0.059666,1.0,0.5927,0
212210,10,0.413633,0.50,0.500000,0.023866,1.0,-0.1263,10,0.472342,0.500000,0.500000,0.023866,0.0,0.1779,0


In [ ]:
X = train.iloc[:,:-1].values
y = train.iloc[:,-1].values

from sklearn.model_selection import train_test_split
X_train,X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2,shuffle = True, stratify = y) 
print("Test Len:",len(X_test)," ",len(y_test))

Test Len: 42443   42443


In [ ]:
train

,review_len_x,Rn_x,Rp_x,Rs_x,Rc_x,Rd_x,Rsc_x,review_len_y,Rn_y,Rp_y,Rs_y,Rc_y,Rd_y,Rsc_y,target
0,5,0.544357,0.52,0.823333,0.006494,0.0,0.0000,4,0.233709,0.300000,0.616667,0.005195,1.0,0.0000,1
1,5,0.544357,0.52,0.823333,0.006494,0.0,0.0000,14,0.673415,-0.100000,0.400000,0.016883,1.0,0.0000,1
2,5,0.544357,0.52,0.823333,0.006494,0.0,0.0000,5,0.319747,0.600000,0.800000,0.006494,1.0,0.4215,1
3,5,0.544357,0.52,0.823333,0.006494,0.0,0.0000,2,0.546925,0.700000,0.600000,0.002597,0.0,0.4404,1
4,5,0.544357,0.52,0.823333,0.006494,0.0,0.0000,2,0.546925,0.700000,0.600000,0.002597,0.0,0.4404,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212207,10,0.413633,0.50,0.500000,0.023866,1.0,-0.1263,15,0.689470,0.700000,0.600000,0.035800,1.0,0.1531,0
212208,10,0.413633,0.50,0.500000,0.023866,1.0,-0.1263,7,0.392391,0.316667,0.600000,0.016706,0.0,0.2382,0
212209,10,0.413633,0.50,0.500000,0.023866,1.0,-0.1263,25,0.575335,0.233333,0.255556,0.059666,1.0,0.5927,0
212210,10,0.413633,0.50,0.500000,0.023866,1.0,-0.1263,10,0.472342,0.500000,0.500000,0.023866,0.0,0.1779,0
